In [1]:
import os
import shutil
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from openpyxl import Workbook

Initialize a workbook

In [2]:
wb = Workbook()
#Get currently active sheet
ws = wb.active
ws.cell(column=1, row=3, value='Standard Deviation')
ws.cell(column=1, row=4, value='Maximum')
ws.cell(column=1, row=5, value='Mean')

<Cell 'Sheet'.A5>

Fetching all the trace files and storing them in two variables. One of them include trace files of Simulation for PTP with isolated axis at 10% velocity, while the other two variables include trace files of Real robot.

In [3]:
# Fetch all regular motion files
target_path_1 = '../../Traces_Real/PTP_Vel'
target_path_2 = '../../Traces_Simulation/PTP_vel_new'

root_path = './New'
fileNames_1 = os.listdir(target_path_1)
fileNames_2 = os.listdir(target_path_2)

fileNames_Real = [file for file in fileNames_1 if '50_po85_KRCIpo.csv' in file]
fileNames_Sim = [file for file in fileNames_2 if '50_KRCIpo.csv' in file]

for file in fileNames_Real:
    shutil.copy(os.path.join(target_path_1, file), root_path)

for file in fileNames_Sim:
    shutil.copy(os.path.join(target_path_2, file), root_path)

Standard Deviation between real robot and simulation for PTP with isolated axis at 10% velocity

In [4]:
#Standard Deviation of PTP with isolated axis with 10% velocity

li_Sim = []
li_Real =[]
if (len(fileNames_Sim)==len(fileNames_Real)):
    for i in range (len(fileNames_Sim)):

        sim = root_path + '/' + fileNames_Sim[i]
        real = root_path + '/' + fileNames_Real[i]

        # Read csv
        df_Sim = pd.read_csv(sim, delimiter=';', encoding='cp1252')
        df_Real = pd.read_csv(real, delimiter=';', encoding='cp1252')
        li_Sim.append(df_Sim)
        li_Real.append(df_Real)

        # Scaling
        scalar = MinMaxScaler()
        scaled_Sim = scalar.fit_transform(li_Sim[i])
        scaled_Real = scalar.fit_transform(li_Real[i])
        scaled_Sim_df = pd.DataFrame(scaled_Sim, columns=li_Sim[i].columns)
        scaled_Real_df = pd.DataFrame(scaled_Real, columns=li_Real[i].columns)

        #Calculate standard deviation and the Maximum difference
        pos_col = scaled_Sim_df.columns[i+7]
        diff = np.array([abs(scaled_Real_df[pos_col] - scaled_Sim_df[pos_col])])
        std = np.nanstd(diff)
        max = np.nanmax(diff)
        mean = np.nanmean(diff)
        print('Standard deviation of scaled difference for Axis ' + str(i+1) + ' = ' + str(std))
        print('Maximum of scaled difference for Axis ' + str(i+1) + ' = ' + str(max))
        print('Mean of scaled difference for Axis ' + str(i+1) + ' = ' + str(mean))
        
        # Write to excel
        ws.cell(column=i+2, row=1, value='Axis'+str(i+1))
        ws.cell(column=i+2, row=3, value=std)
        ws.cell(column=i+2, row=4, value=max)
        ws.cell(column=i+2, row=5, value=mean)

#Save the Workbook
wb.save(filename='Standard_Deviation and Max_50.xlsx')

Standard deviation of scaled difference for Axis 1 = 0.0014494363697131179
Maximum of scaled difference for Axis 1 = 0.004188062556162553
Mean of scaled difference for Axis 1 = 0.0024976802414420083
Standard deviation of scaled difference for Axis 2 = 0.0022648546056742965
Maximum of scaled difference for Axis 2 = 0.0062658782667179835
Mean of scaled difference for Axis 2 = 0.0027325335084786664
Standard deviation of scaled difference for Axis 3 = 0.0024193337767834232
Maximum of scaled difference for Axis 3 = 0.007235890714274662
Mean of scaled difference for Axis 3 = 0.0035613062444777256
Standard deviation of scaled difference for Axis 4 = 0.0006117199045281294
Maximum of scaled difference for Axis 4 = 0.0022538699195621126
Mean of scaled difference for Axis 4 = 0.0014187324434956812
Standard deviation of scaled difference for Axis 5 = 0.001146302334935073
Maximum of scaled difference for Axis 5 = 0.004308667295498558
Mean of scaled difference for Axis 5 = 0.0021082872285749666
Stan